In [4]:
import tensorflow as tf
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np 

In [5]:
df=pd.read_csv("data.csv",parse_dates=True)
print(df.head())

   Unnamed: 0            date time  Free  month  day  hour  minu  weekday
0           0  2011-11-28 00:00:00     0     11   28     0     0        0
1           1  2011-11-28 00:15:00     0     11   28     0    15        0
2           2  2011-11-28 00:30:00     0     11   28     0    30        0
3           3  2011-11-28 00:45:00     0     11   28     0    45        0
4           4  2011-11-28 01:00:00     0     11   28     1     0        0


In [58]:
x_axis=df[['month','day','hour','minu','weekday']]
y_axis=df[['Free']]


x_train, x_test, y_train1, y_test1 = train_test_split(x_axis, y_axis, test_size=0.1)

y_train = pd.DataFrame({'0': [], '1': []})
for i  in range(0,len(y_train1)):
    z=[0,0]
    if(y_train1.iloc[i].Free==0):
        z[0]=1
    else:
        z[1]=1
    
    y_train = y_train.append({'0':int(z[0]), '1':int(z[1])}, ignore_index=True)


y_test = pd.DataFrame({'0': [], '1': []})
for i  in range(0,len(y_test1)):
    z=[0,0]
    if(y_test1.iloc[i].Free==0):
        z[0]=1
    else:
        z[1]=1
    
    y_test = y_test.append({'0':int(z[0]), '1':int(z[1])}, ignore_index=True)

print(y_train.head())
print(y_test.head())
       

     0    1
0  1.0  0.0
1  1.0  0.0
2  0.0  1.0
3  1.0  0.0
4  0.0  1.0
     0    1
0  0.0  1.0
1  1.0  0.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0


In [63]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 2
batch_size = 100

x = tf.placeholder('float', [None, len(x_train.iloc[0])])
y = tf.placeholder('float',[None, len(y_train.iloc[0])])

def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([len(x_train.iloc[0]), n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}


    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']
    
    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    # OLD VERSION:
    #cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    # NEW:
   
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        # OLD:
        #sess.run(tf.initialize_all_variables())
        # NEW:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            i=0
            while(i<len(x_train)):
                start=i
                end=i+ batch_size
                
                batch_x=np.array(x_train[start:end])
                batch_y=np.array(y_train[start:end])                
                
                
                _, c = sess.run([optimizer, cost], feed_dict={x:batch_x, y: batch_y})
                epoch_loss += c
                i=i+batch_size
            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:x_test, y:y_test}))

train_neural_network(x)

Epoch 0 completed out of 10 loss: 132897.60888671875
Epoch 1 completed out of 10 loss: 59469.7158203125
Epoch 2 completed out of 10 loss: 38568.396484375
Epoch 3 completed out of 10 loss: 26220.798278808594
Epoch 4 completed out of 10 loss: 14132.06704711914
Epoch 5 completed out of 10 loss: 11827.139373779297
Epoch 6 completed out of 10 loss: 10845.101440429688
Epoch 7 completed out of 10 loss: 11076.003356933594
Epoch 8 completed out of 10 loss: 12347.807312011719
Epoch 9 completed out of 10 loss: 14494.199493408203
Accuracy: 0.6736111
